# Text Classification for Topic Modelling

## Notebook Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import nltk

from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer, SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
nltk.download('stopwords')
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [3]:
from nltk.corpus import wordnet
! unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

## Data Loading

In [4]:
train_df = pd.read_csv('/kaggle/input/topic-modeling-for-research-articles/Train.csv')
train_df.head()

,id,ABSTRACT,Computer Science,Mathematics,Physics,Statistics,Analysis of PDEs,Applications,Artificial Intelligence,Astrophysics of Galaxies,...,Methodology,Number Theory,Optimization and Control,Representation Theory,Robotics,Social and Information Networks,Statistics Theory,Strongly Correlated Electrons,Superconductivity,Systems and Control
0,1824,a ever-growing datasets inside observational a...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3094,we propose the framework considering optimal $...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8463,nanostructures with open shell transition meta...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,2082,stars are self-gravitating fluids inside which...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8687,deep neural perception and control networks ar...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14004 entries, 0 to 14003
Data columns (total 31 columns):
 #   Column                                        Non-Null Count  Dtype 
---  ------                                        --------------  ----- 
 0   id                                            14004 non-null  int64 
 1   ABSTRACT                                      14004 non-null  object
 2   Computer Science                              14004 non-null  int64 
 3   Mathematics                                   14004 non-null  int64 
 4   Physics                                       14004 non-null  int64 
 5   Statistics                                    14004 non-null  int64 
 6   Analysis of PDEs                              14004 non-null  int64 
 7   Applications                                  14004 non-null  int64 
 8   Artificial Intelligence                       14004 non-null  int64 
 9   Astrophysics of Galaxies                      14004 non-null  int64 
 10

In [6]:
sample_text = train_df['ABSTRACT'][0]
sample_text

'a ever-growing datasets inside observational astronomy have challenged scientists inside many aspects, including an efficient and interactive data exploration and visualization. many tools have been developed to confront this challenge. however, they usually focus on displaying a actual images or focus on visualizing patterns within catalogs inside the predefined way. inside this paper we introduce vizic, the python visualization library that builds a connection between images and catalogs through an interactive map of a sky region. vizic visualizes catalog data over the custom background canvas with the help of a shape, size and orientation of each object inside a catalog. a displayed objects inside a map are highly interactive and customizable comparing to those inside a images. these objects should be filtered by or colored by their properties, such as redshift and magnitude. they also should be sub-selected with the help of the lasso-like tool considering further analysis with the

## Text Preprocessing

In [7]:
def preprocess_text(text: str, stopwords=set(stopwords.words('english')), wnl=WordNetLemmatizer()) -> list:
    # Tokenising
    words = word_tokenize(text.lower())
    
    # Removing Stopwords & Punctuation
    filtered_text = [word for word in words if word not in stopwords and word.isalpha()]
    
    # Lemmatization
    preprocessed_text = [wnl.lemmatize(word) for word in filtered_text]
    
    return preprocessed_text

In [8]:
nested_list = train_df.ABSTRACT.apply(preprocess_text)
nested_list.head()

0    [datasets, inside, observational, astronomy, c...
1    [propose, framework, considering, optimal, exc...
2    [nanostructures, open, shell, transition, meta...
3    [star, fluid, inside, pressure, buoyancy, rota...
4    [deep, neural, perception, control, network, l...
Name: ABSTRACT, dtype: object

In [9]:
topics = [column for column in train_df.columns if column != 'id' and column != 'ABSTRACT']
topics

['Computer Science',
 'Mathematics',
 'Physics',
 'Statistics',
 'Analysis of PDEs',
 'Applications',
 'Artificial Intelligence',
 'Astrophysics of Galaxies',
 'Computation and Language',
 'Computer Vision and Pattern Recognition',
 'Cosmology and Nongalactic Astrophysics',
 'Data Structures and Algorithms',
 'Differential Geometry',
 'Earth and Planetary Astrophysics',
 'Fluid Dynamics',
 'Information Theory',
 'Instrumentation and Methods for Astrophysics',
 'Machine Learning',
 'Materials Science',
 'Methodology',
 'Number Theory',
 'Optimization and Control',
 'Representation Theory',
 'Robotics',
 'Social and Information Networks',
 'Statistics Theory',
 'Strongly Correlated Electrons',
 'Superconductivity',
 'Systems and Control']

In [10]:
topic_ids = {}

for topic in topics:
    ids = train_df[train_df[topic] == 1].index
    nested_list_topic = nested_list.loc[ids]
    topic_ids[topic] = nested_list_topic
    
    flat_list = [item for sublist in nested_list_topic for item in sublist]
    topic_ids[topic]['all_words'] = flat_list

In [11]:
topic_ids

{'Computer Science': 1            [propose, framework, considering, optimal, exc...
 4            [deep, neural, perception, control, network, l...
 5            [analyzing, job, hopping, behavior, important,...
 8            [nowadays, data, compressor, applied, many, pr...
 10           [study, problem, extracting, selective, connec...
                                    ...                        
 13998        [paper, summarises, analysis, recognition, tra...
 13999        [methodology, automatic, detection, event, bas...
 14000        [consider, case, inside, robot, navigate, insi...
 14002        [present, framework, implementation, relying, ...
 all_words    [propose, framework, considering, optimal, exc...
 Name: ABSTRACT, Length: 5885, dtype: object,
 'Mathematics': 8            [nowadays, data, compressor, applied, many, pr...
 12           [show, embedding, inside, euclidean, space, ba...
 15           [inside, proved, every, riemannian, manifold, ...
 16           [fitzpatr